In [ ]:
# PRACTICAL 06: Ratio-to-Moving Average Seasonal Indices
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Load dataset
path = "Symphony-Data.csv"  # Update with your actual path
df = pd.read_csv(path)

# Step 2: Preprocessing
df = df.drop(columns=['OPEN', 'HIGH', 'LOW', 'VOLUME', 'CHANGE(%)'])
df['DATE'] = pd.to_datetime(df['DATE'], format="%d-%b-%y")
df.sort_values('DATE', inplace=True)

# Step 3: Extract Month and Year
df['Month'] = df['DATE'].dt.month
df['Year'] = df['DATE'].dt.year

# Step 4: Calculate monthly average for each year
monthly_avg = df.groupby(['Month', 'Year'])['PRICE'].mean().unstack()

# Step 5: Calculate overall monthly average
monthly_avg['Monthly_Avg'] = monthly_avg.mean(axis=1)

# Step 6: Calculate centered moving average
monthly_avg['Centered_MA'] = monthly_avg['Monthly_Avg'].rolling(window=2, center=True).mean()

# Step 7: Calculate ratio of actual to centered MA
monthly_avg['Ratio'] = monthly_avg['Monthly_Avg'] / monthly_avg['Centered_MA']

# Step 8: Normalize ratios to sum = 12 (months)
sum_ratios = monthly_avg['Ratio'].sum()
monthly_avg['Seasonal_Index'] = monthly_avg['Ratio'] * (12 / sum_ratios)

# Step 9: Deseasonalize yearly data using seasonal index
for year in [2019, 2020, 2021, 2022, 2023, 2024]:
    monthly_avg[f'Deseasonalized_{year}'] = monthly_avg[year] / monthly_avg['Seasonal_Index']

# Step 10: Plotting
plt.figure(figsize=(14, 10))

# Original Data
plt.subplot(3, 1, 1)
for year in [2019, 2020, 2021, 2022, 2023, 2024]:
    plt.plot(monthly_avg.index, monthly_avg[year], marker='o', label=str(year))
plt.title("Original Monthly Stock Prices")
plt.xlabel("Month")
plt.ylabel("Price")
plt.legend(loc='upper right')
plt.grid()

# Seasonal Indices
plt.subplot(3, 1, 2)
plt.plot(monthly_avg.index, monthly_avg['Seasonal_Index'], marker='o', color='orange', label="Seasonal Index")
plt.title("Seasonal Indices (Ratio-to-MA Method)")
plt.xlabel("Month")
plt.ylabel("Index")
plt.legend(loc='upper right')
plt.grid()

# Deseasonalized Data
plt.subplot(3, 1, 3)
for year in [2019, 2020, 2021, 2022, 2023, 2024]:
    plt.plot(monthly_avg.index, monthly_avg[f'Deseasonalized_{year}'], marker='o', label=f"{year} Deseasonalized")
plt.title("Deseasonalized Monthly Prices")
plt.xlabel("Month")
plt.ylabel("Price")
plt.legend(loc='upper right')
plt.grid()

plt.tight_layout()
plt.show()
